In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import time

In [2]:
df = pd.read_csv('ToN-IoT_balanced_smote.csv')

In [3]:
df

,src_port,dst_port,duration,src_bytes,dst_bytes,missed_bytes,src_ip_bytes,dst_ip_bytes,dns_rcode,label,...,weird_name_dnp3_corrupt_header_checksum,weird_name_inappropriate_FIN,weird_name_possible_split_routing,weird_addl_43,weird_addl_46,weird_addl_48,weird_addl_None,weird_notice_F,weird_notice_None,type
0,0.101952,3.799303,175.726962,0.152327,3.438007,0.0,86.672008,0.001470,0.0,0.0,...,False,False,False,False,False,False,True,True,False,5.0
1,2.558822,1.118792,0.000000,0.000000,0.000000,0.0,0.000366,0.000000,0.0,0.0,...,False,False,False,False,False,False,True,False,True,5.0
2,0.101790,3.713672,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,False,False,False,False,False,False,True,True,False,5.0
3,1.856906,0.753177,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,False,False,False,False,False,False,True,False,True,5.0
4,2.523521,0.003801,0.000001,0.000000,0.000024,0.0,0.000000,0.002205,0.0,0.0,...,False,False,False,False,False,False,True,True,False,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,3.279039,0.005737,0.000000,0.000000,0.000000,0.0,0.000302,0.000000,0.0,1.0,...,False,False,False,False,False,False,True,False,True,9.0
2999996,1.823237,0.003801,0.000024,0.000007,0.000009,0.0,0.000791,0.001046,0.0,1.0,...,False,False,False,False,False,False,True,False,True,9.0
2999997,2.563471,0.005737,0.000000,0.000000,0.000000,0.0,0.000302,0.000000,0.0,1.0,...,False,False,False,False,False,False,True,False,True,9.0
2999998,1.853248,0.005737,0.000000,0.000000,0.000000,0.0,0.000302,0.000000,0.0,1.0,...,False,False,False,False,False,False,True,False,True,9.0


In [4]:
X = df.drop(['type'], axis=1)
y = df['type']
# one hot encoding
y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# convert to numpy array
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# define the model
model = Sequential()
model.add(Bidirectional(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional((LSTM(64))))
model.add(Dropout(0.1))
model.add((Dense(10, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='lstm_best_model.h5', verbose=1, save_best_only=True, monitor = 'val_accuracy', mode = 'max')
start = time.time()
# train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[checkpointer])
end = time.time()
print(end - start)



Epoch 1/10


37500/37500 [==============================] - ETA: 0s - loss: 0.3846 - accuracy: 0.8674
Epoch 1: val_accuracy improved from -inf to 0.90314, saving model to lstm_best_model.h5
37500/37500 [==============================] - 10346s 276ms/step - loss: 0.3846 - accuracy: 0.8674 - val_loss: 0.2646 - val_accuracy: 0.9031
Epoch 2/10


C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


37500/37500 [==============================] - ETA: 0s - loss: 0.2213 - accuracy: 0.9266
Epoch 2: val_accuracy improved from 0.90314 to 0.94325, saving model to lstm_best_model.h5
37500/37500 [==============================] - 7727s 206ms/step - loss: 0.2213 - accuracy: 0.9266 - val_loss: 0.1828 - val_accuracy: 0.9432
Epoch 3/10
37500/37500 [==============================] - ETA: 0s - loss: 0.1499 - accuracy: 0.9521
Epoch 3: val_accuracy improved from 0.94325 to 0.95685, saving model to lstm_best_model.h5
37500/37500 [==============================] - 5133s 137ms/step - loss: 0.1499 - accuracy: 0.9521 - val_loss: 0.1323 - val_accuracy: 0.9569
Epoch 4/10
37500/37500 [==============================] - ETA: 0s - loss: 0.1239 - accuracy: 0.9611
Epoch 4: val_accuracy improved from 0.95685 to 0.97132, saving model to lstm_best_model.h5
37500/37500 [==============================] - 5215s 139ms/step - loss: 0.1239 - accuracy: 0.9611 - val_loss: 0.0947 - val_accuracy: 0.9713
Epoch 5/10
37500/3

In [5]:
model = load_model('lstm_best_model.h5')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print("Time taken to predict: ", end-start)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

18750/18750 [==============================] - 458s 24ms/step
Time taken to predict:  464.45312452316284
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60144
           1       0.97      0.97      0.97     60079
           2       1.00      0.97      0.98     60047
           3       0.98      0.95      0.97     59974
           4       0.96      0.99      0.98     60175
           5       1.00      1.00      1.00     59762
           6       0.99      0.98      0.98     59631
           7       0.99      0.99      0.99     59909
           8       0.98      0.97      0.98     59992
           9       0.95      0.99      0.97     60287

    accuracy                           0.98    600000
   macro avg       0.98      0.98      0.98    600000
weighted avg       0.98      0.98      0.98    600000

0.9813766666666667


In [6]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Accuracy of each class:')
for i in range(cm.shape[0]):
    print(i, cm[i][i])

Accuracy of each class:
0 0.9995677041766428
1 0.9740674778208692
2 0.9670591370093427
3 0.9505118884850102
4 0.9944495222268384
5 0.9999163347946856
6 0.9753483926145796
7 0.9921046921163765
8 0.9673456460861448
9 0.9933153084412891
